In [1]:
import sca as sca
import numpy as np
import sklearn as sk
import scipy as sp
import matplotlib.pyplot as plt
import time
from nilearn import datasets
from nilearn import image
from nilearn.input_data import NiftiMasker
from nilearn import plotting

In [2]:
adhd_dataset = datasets.fetch_adhd(n_subjects=1)
epi_filename = adhd_dataset.func[0]

/home/pbellec/env/jupyter_ml/lib/python3.6/site-packages/nilearn/datasets/func.py:503: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  dtype=None)


In [3]:
masker = NiftiMasker(mask_strategy='epi', detrend=True, standardize=True, smoothing_fwhm=2)
masker.fit(epi_filename)

NiftiMasker(detrend=True, dtype=None, high_pass=None, low_pass=None,
      mask_args=None, mask_img=None, mask_strategy='epi',
      memory=Memory(location=None), memory_level=1, sample_mask=None,
      sessions=None, smoothing_fwhm=2, standardize=True, t_r=None,
      target_affine=None, target_shape=None, verbose=0)

In [4]:
y = masker.transform(epi_filename)
y.shape

(176, 62546)

In [32]:
t0 = time.clock()
stab_maps = sca.recursive_cluster(y.transpose(), n_clusters=10, n_states=20, 
                                  subsample_size=50, n_replications=20, max_iter=30, contiguous=True)
print(time.clock()-t0, "seconds process time")

209.98358799999994 seconds process time


In [33]:
stab_maps_img = masker.inverse_transform(stab_maps.transpose())

In [56]:
rsn = image.index_img(stab_maps_img, 10)
plotting.view_img(rsn, symmetric_cmap=True, vmax=1, threshold=0.1)

/home/pbellec/env/jupyter_ml/lib/python3.6/site-packages/nilearn/plotting/js_plotting_utils.py:103: UserWarning: It seems you have created more than 10 nilearn views. As each view uses dozens of megabytes of RAM, you might want to delete some of them.
  warnings.warn('It seems you have created more than 10 '
